In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import glob

### Create some code for dictionaries to map countries and parties names

In [2]:
countries_dict = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "DE": "Germany",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
    "UK": "United Kingdom"
};

political_group_map= {
"GP1001": "EPP-ED",
 "GP1002": "PES",
 "GP1003": "ALDE",
 "GP1004": "G-EFA",
 "GP1005": "EUL-NGL",
 "GP1006": "UEN",
 "GP1007": "IND-DEM",
 "IND": "Independent",
"GP1201": "EPP-ED",
 "GP1202": "PES",
 "GP1203": "ALDE",
 "GP1204": "G-EFA",
 "GP1205": "EUL-NGL",
 "GP1206": "UEN",
 "GP1207": "IND-DEM",
 "IND": "Independent",
"GP1401": "EPP-ED",
 "GP1402": "S&D",
 "GP1403": "ECR",
 "GP1404": "ALDE",
 "GP1405": "GUE/NGL",
 "GP1406": "G-EFA",
 "GP1407": "EFDD",
 "IND": "Independent",
'EPP':'EPP',
'SD':'S&D',
'ALDE': 'ALDE',
'GUENGL':'GUE/NGL',
'GREENSEFA': "G-EFA",
'ENF':'ENF',
'ECR':'ECR'
}

party_name_map = {
    "EPP-ED": "European People's Party–European Democrats",
    "PES": "Party of European Socialists",
    "ALDE": "Alliance of Liberals and Democrats for Europe",
    "G-EFA": "The Greens–European Free Alliance",
    "EUL-NGL": "European United Left–Nordic Green Left",
    "UEN": "Union for Europe of the Nations",
    "IND-DEM": "Independence and Democracy",
    "Independent": "(Independent Members)",

    # Additional, based on 2014-2019 
    "EPP": "European People's Party",
    "S&D": "Progressive Alliance of Socialists and Democrats",
    "ECR": "European Conservatives and Reformists",
    "GUE/NGL": "European United Left–Nordic Green Left",
    "EFDD": "Europe of Freedom and Direct Democracy",
    "ENF": "Europe of Nations and Freedom",
}


### Read all files by election and country

Data is sourced from the official European parliament website (https://results.elections.europa.eu/en/tools/download-datasheets/)

In [3]:


# Elections years/folders
years=['2004-2009','2009-2014','2014-2019','2019-2024']

i=0#just a counter so that the data is created before appending
for period in years:
    if i==0:
        csv_files=(glob.glob('./data/EU/'+period+'/*.{}'.format('csv')))
        i=1
    else:
        csv_files+=(glob.glob('./data/EU/'+period+'/*.{}'.format('csv')))

#csv_files



### Create the eu_df dataframe

In [4]:
df=pd.DataFrame()


for file in csv_files:
    df1=pd.read_csv(file, sep=';')
    country=file.replace('.csv','')[-3:].replace('\\','')
    if country!='eu':
        #Define election year
        df1['Election']=file[10:14]
        #Define country code in capital letters
        df1['Country']=country.upper()
        #Map country code to country name
        df1['Country_name']=df1['Country'].map(countries_dict,na_action='ignore')
        df=pd.concat([df,df1],axis=0,ignore_index=True)

#Keep only useful columns
eu_df=df[['Election','GROUP_ID','SEATS_TOTAL','SEATS_PERCENT_EU','Country','Country_name']]

#keep only EU countries
eu_df=eu_df.dropna(subset='Country_name').sort_values(by='Election').reset_index().drop('index',axis='columns')

eu_df.GROUP_ID=eu_df.GROUP_ID.fillna("IND")
eu_df['Party_short']=eu_df.GROUP_ID.map(political_group_map)
eu_df['Party_name']=eu_df['Party_short'].map(party_name_map)




In [5]:
eu_df.head()

,Election,GROUP_ID,SEATS_TOTAL,SEATS_PERCENT_EU,Country,Country_name,Party_short,Party_name
0,2004,GP1001,6,0.82,AT,Austria,EPP-ED,European People's Party–European Democrats
1,2004,IND,0,0.00,LU,Luxembourg,Independent,(Independent Members)
2,2004,GP1001,3,0.41,LV,Latvia,EPP-ED,European People's Party–European Democrats
3,2004,GP1002,0,0.00,LV,Latvia,PES,Party of European Socialists
4,2004,GP1003,1,0.14,LV,Latvia,ALDE,Alliance of Liberals and Democrats for Europe


In [6]:
eu_df_party=pd.DataFrame(eu_df.groupby(by=['Election','Party_name']).SEATS_TOTAL.sum())
eu_df_party.sort_values(by=['Election','SEATS_TOTAL'],ascending=[True,False])

SEATS_TOTAL
Election Party_name                                                   
2004     European People's Party–European Democrats                268
         Party of European Socialists                              200
         Alliance of Liberals and Democrats for Europe              88
         The Greens–European Free Alliance                          42
         European United Left–Nordic Green Left                     41
         Independence and Democracy                                 37
         (Independent Members)                                      29
         Union for Europe of the Nations                            27
2009     European People's Party–European Democrats                265
         Party of European Socialists                              184
         Alliance of Liberals and Democrats for Europe              84
         The Greens–European Free Alliance                          55
         European United Left–Nordic Green Left                     54
         Union for Europe of the Nations                            35
         Independence and Democracy                                 32
         (Independent Members)                                      27
2014     European People's Party–European Democrats                221
         Progressive Alliance of Socialists and Democrats          191
         European Conservatives and Reformists                      70
         Alliance of Liberals and Democrats for Europe              67
         (Independent Members)                                      52
         European United Left–Nordic Green Left                     52
         The Greens–European Free Alliance                          50
         Europe of Freedom and Direct Democracy                     48
2019     European People's Party                                   182
         Progressive Alliance of Socialists and Democrats          154
         Alliance of Liberals and Democrats for Europe             108
         The Greens–European Free Alliance                          74
         Europe of Nations and Freedom                              73
         European Conservatives and Reformists                      62
         (Independent Members)                                      57
         European United Left–Nordic Green Left                     41

In [34]:
country_code='IT'

country_df=eu_df.loc[eu_df['Country']==country_code]
pd.DataFrame(country_df.groupby(by=['Election','Party_name'])['SEATS_TOTAL'].sum()).sort_values(by=['Election','SEATS_TOTAL'],ascending=[True,False])
#country_df

SEATS_TOTAL
Election Party_name                                                   
2004     European People's Party–European Democrats                 24
         Party of European Socialists                               16
         Alliance of Liberals and Democrats for Europe              12
         Union for Europe of the Nations                             9
         European United Left–Nordic Green Left                      7
         (Independent Members)                                       4
         Independence and Democracy                                  4
         The Greens–European Free Alliance                           2
2009     European People's Party–European Democrats                 35
         Party of European Socialists                               21
         Independence and Democracy                                  9
         Alliance of Liberals and Democrats for Europe               7
         (Independent Members)                                       0
         European United Left–Nordic Green Left                      0
         The Greens–European Free Alliance                           0
         Union for Europe of the Nations                             0
2014     Progressive Alliance of Socialists and Democrats           31
         Europe of Freedom and Direct Democracy                     17
         European People's Party–European Democrats                 17
         (Independent Members)                                       5
         European United Left–Nordic Green Left                      3
         Alliance of Liberals and Democrats for Europe               0
         European Conservatives and Reformists                       0
         The Greens–European Free Alliance                           0
2019     Europe of Nations and Freedom                              28
         Progressive Alliance of Socialists and Democrats           19
         (Independent Members)                                      14
         European People's Party                                     7
         European Conservatives and Reformists                       5